# Getting Started: Basic System Properties

Here, we present the basic properties of the LAMMPS-derived classes that will be later used for the implicit derivative calculations.

Currently, only the **linear-in-descriptor SNAP potentials** are used.

In [1]:
# Standard libraries
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt

# No parallel MPI runs in the notebook, but keep comm for consistency
comm = None

### Import the package classes and functions

In [2]:
from lammps_implicit_der import SNAP, LammpsImplicitDer
from lammps_implicit_der.systems import BCC, BCC_BINARY, BCC_BINARY_VACANCY, BCC_VACANCY, BCC_SIA, FromData, HCP
from lammps_implicit_der.tools import plot_tools

### Create a BCC system

As an instance of the `BCC` class. `BCC` is a child class of `LammpsImplicitDer`.

In [3]:
bcc_pure = BCC(alat=3.18, ncell_x=2, minimize=False,
               logname='bcc.log', lmp_cmd_filename='bcc_commands.lammps',
               snapcoeff_filename='W_REF.snapcoeff', verbose=True, comm=comm)


--------------------------------------------------------------------------------
Running LAMMPS with the following arguments:
-screen none -log bcc.log

Setting SNAP potential

                  SNAP coefficients for: W
                          quadraticflag: 0
 Number of parameters (excluding beta0): 55
                                Element:  W  |  R =  0.5000 w =  1.0000

Number of atoms: 16, largest force value: 6.051e-15, force norm: 2.335e-14


Here we used the following input arguments:

* `alat`: lattice parameter in Angstroms
* `ncell_x`: number of BCC unit cells in the x-direction. If `ncell_y` and `ncell_z` are not specified, they will be equal to `ncell_x`
* `minimize`: minimize the atomic positions
* `logname`: LAMMPS log file name
* `lmp_command_filename`: filename containing all the LAMMPS commands sent to LAMMPS for an object
* `snapcoeff_filename`: filename of the SNAP coefficients file. If `snapparam_filename` is not specified, it will be guessed by changing the `.snapcoeff` suffix to `.snapparam`
* `verbose`: output the system information
* `comm`: MPI communicator for parallel runs, here it is `None`. To initialize a communicator add to the script (requires `mpi4py`):
```python
from lammps_implicit_der.tools import mpi_print, initialize_mpi
comm, rank = initialize_mpi()
```

Refer to the docstring documentation of the `LammpsImplicitDer` class (`./lammps_implicit_der/lmp_der/implicit_der.py` file) and `BCC` child class (`./lammps_implicit_der/systems/bcc_lattices.py` file) for the full list of input parameters.

### Lammps object within the class

Under the hood, at the initialization stage, the `BCC` class creates a LAMMPS instance:

```python
self.lmp = lammps(cmdargs=self.cmdargs, comm=self.comm)
```

Working with `BCC` or any child class consists in sending commands and data to and receiving data from LAMMPS at runtime. All the commands sent to lammps will be stored in the `lmp_command_filename` file. The LAMMPS logs will be printed out in the `logname` file.

### Manually send a command to LAMMPS

In [4]:
bcc_pure.lmp_commands_string("run 0")

### Save the system to a LAMMPS data file

In [5]:
bcc_pure.write_data('bcc_pure.data')

# First 12 lines of the data file
with open('bcc_pure.data', 'r') as f:
    for i in range(12):
        print(f.readline().strip())
print('...')

LAMMPS data file via write_data, version 2 Aug 2023, timestep = 0, units = metal

16 atoms
1 atom types

0 6.36 xlo xhi
0 6.36 ylo yhi
0 6.36 zlo zhi

Masses

1 183.84
...


### Save the coordinates of the systems to a .xyz file

In [6]:
bcc_pure.write_xyz_file('bcc_pure.xyz')

# First 5 lines of the xyz file
with open('bcc_pure.xyz', 'r') as f:
    for i in range(5):
        print(f.readline().strip())
print('...')

16
Atomic coordinates generated by LammpsImplicitDer
W 0.031800 0.031800 0.031800
W 1.621800 1.621800 1.621800
W -3.148200 0.031800 0.031800
...


## Properties



In [7]:
print(f'Number of atoms: {bcc_pure.Natom}')
print(f'Number of SNAP descriptors: {bcc_pure.Ndesc}')

Number of atoms: 16
Number of SNAP descriptors: 55


### Coordinates and minimum image convention

Coordinates can be accessed as `self.X_coord`, which is a flattened array of positions, so it has a shape of `(3 * Natom)`

In [8]:
#Retrieve the coordinates of the atoms
X_coord = bcc_pure.X_coord.copy()
print(X_coord.shape)

# Rehape the coordinates
X_coord_3D = X_coord.reshape(-1, 3)
print(f'Coordinates of 3rd atom: {X_coord_3D[2]}')

(48,)
Coordinates of 3rd atom: [-3.1482  0.0318  0.0318]


### Minimum image convention is automatically applied when `self.X_coord` is assigned with a new array 

In [9]:
# Assign a very large coordinate to check the minimum image convention
X_coord_test = X_coord.copy()
X_coord_test[5] = 5000
bcc_pure.X_coord = X_coord_test
print(f'{bcc_pure.X_coord[5]:.3f}')
bcc_pure.X_coord = X_coord

1.040


Note that in the cell above, we changed `X_coord` and applied the minimum image convention. However, these coordinates were not sent to LAMMPS. To do that, one has to manually scatter the coordinates: 

In [10]:
bcc_pure.scatter_coord()

### Simulation box

In [11]:
cell = bcc_pure.cell.copy()
print(f'{cell}')

[[6.36 0.   0.  ]
 [0.   6.36 0.  ]
 [0.   0.   6.36]]


### Energy

Can be accessed as `self.energy`. The energy is _updated_ every time (with LAMMPS `thermo_pe` command) the attribute is accessed. 

In [12]:
print(f'Energy of the system (eV): {bcc_pure.energy:.6f}')

Energy of the system (eV): -89.055528


### Forces

$$ \nabla_X U(\mathbf{X}, \mathbf{\Theta}) \in \mathbb{R}^{3N}$$

In [13]:
force_array = bcc_pure.compute_forces()
print(f'Forces shape: {force_array.shape}')

Forces shape: (48,)


### Virial
6 virial components per atom, per descriptor: $V_{xx}, V_{yy}, V_{zz}, V_{yz}, V_{xz}, V_{xy}$

In [22]:
bcc_pure.gather_virial()
print(f'Number of atoms: {bcc_pure.Natom}')
print(f'Number of SNAP descriptors: {bcc_pure.Ndesc}')
print(f'Virial tensor shape: {bcc_pure.virial.shape}')

Number of atoms: 16
Number of SNAP descriptors: 55
Virial tensor shape: (16, 6, 55)


### Pressure

In [23]:
bcc_pure.get_pressure_from_virial()
print(f'Pressure: {bcc_pure.pressure:.3f}')

Pressure: 2.670


### Pressure tensor

$\sigma_{xx}, \sigma_{yy}, \sigma_{zz}, \sigma_{yz}, \sigma_{xz}, \sigma_{xy}$

In [24]:
bcc_pure.get_pressure_tensor_from_virial()
print(f'Pressure tensor: {bcc_pure.pressure_tensor}')

Pressure tensor: [2.66982613e+00 2.66982613e+00 2.66982613e+00 9.77285790e-16
 9.56255809e-15 5.70424499e-15]


### Position Hessian

$$\nabla_{\mathbf{X}\mathbf{X}}U(\mathbf{X}, \mathbf{\Theta}) \in \mathbb{R}^{3N\times 3N}$$

Evaluated with a finite-difference scheme

In [14]:
bcc_pure.compute_hessian()
print(f'Number of atoms: {bcc_pure.Natom}')
print(f'Hessian shape: {bcc_pure.hessian.shape}')

Computing the Hessian...


Hessian (full): 100%|██████████| 48/48 [00:00<00:00, 52.93it/s]

Number of atoms: 16
Hessian shape: (48, 48)


### Mixed Hessian

$$\nabla_{\mathbf{\Theta}\mathbf{X}} U(\mathbf{X}, \mathbf{\Theta}) \in \mathbb{R}^{N_D \times 3N}$$

For linear-in-descriptor potentials, there is no need in finite diff. schemes as it can be accessed directly from LAMMPS as derivative of potential energy over the potential parameters.

In [15]:
print(f'Mixed Hessian shape: {bcc_pure.mixed_hessian.shape}')

Mixed Hessian shape: (55, 48)


Instead of specifying the LAMMPS data file, one can also provide an input script text with an `input_scipt` option, e.g., `FromData(input_script=...)`.

# SNAP potential

Currently, only SNAP and quadratic SNAP potentials are supported. Every `LammpsImplicitDer` instance (`BCC`, `HCP`, etc.), has a `.pot` object, instance of a `SNAP` class. All the potential-related infromation can be retrieved from it.

In [16]:
print(f'General potential information: {bcc_pure.pot}')

General potential information: 
                  SNAP coefficients for: W
                          quadraticflag: 0
 Number of parameters (excluding beta0): 55
                                Element:  W  |  R =  0.5000 w =  1.0000



Potential parameters $\mathbf{\Theta}$ are stored in the `Theta_dict` dictionary:

In [17]:
print(f'Keys of Theta_dict: {bcc_pure.pot.Theta_dict.keys()}')
print(f'First 5 Theta parameters of W: {bcc_pure.pot.Theta_dict["W"]["Theta"][:5]}')
print(f'beta0 parameter: {bcc_pure.pot.Theta_dict["W"]["beta0"]}')
print(f'Weights: {bcc_pure.pot.Theta_dict["weights"]}')
print(f'Radii: {bcc_pure.pot.Theta_dict["radii"]}')
print(f'Dictionary of snapparam: {bcc_pure.pot.snapparam_dict}')

Keys of Theta_dict: dict_keys(['W', 'radii', 'weights'])
First 5 Theta parameters of W: [0.0144887  0.05682754 0.2706519  0.03159091 0.51600315]
beta0 parameter: 0.0
Weights: 1.0
Radii: 0.5
Dictionary of snapparam: {'quadraticflag': 0, 'rcutfac': 5.0, 'twojmax': 8, 'rfac0': 0.99363, 'rmin0': 0, 'bzeroflag': 1, 'bnormflag': 0}


### Save parameters to a file

`SNAP` instances (called `pot` in the cell above) can be created separately, not as an attribute of `LammpsImplicitDer` instance. This can be handy to read the SNAP files, modify the potential and write it down into new files, as shown in the example below.

In [18]:
pot = SNAP.from_files(snapcoeff_filename='W_REF.snapcoeff', snapparam_filename='W_REF.snapparam')

# Modify the potential
pot.Theta_dict['W']['Theta'][3] = 7.0

# Save the potential to files
pot.to_files(snapcoeff_filename='W_REF_new.snapcoeff', snapparam_filename='W_REF_new.snapparam', overwrite=True)

Overwriting ./W_REF_new.snapcoeff
Saved SNAP coefficients to ./W_REF_new.snapcoeff
Saved SNAP parameters to ./W_REF_new.snapparam


## Energy minimization

To minimize energy, one can specify `minimize=True` at the creation of a system instance or, equivalently, call a `minimize_energy()` method as shown below for a BCC Vacancy example. 

In [19]:
bcc_vacancy = BCC_VACANCY(alat=3.18, ncell_x=3, minimize=False,
                          snapcoeff_filename='W_REF.snapcoeff', verbose=False, comm=comm)
print(f'Non-minimized energy (eV): {bcc_vacancy.energy:.8f}')

# Minimize the energy
bcc_vacancy.minimize_energy()
print(f'Minimized energy (eV): {bcc_vacancy.energy:.8f}')

# Minimize energy at creation of an object
bcc_vacancy2 = BCC_VACANCY(alat=3.18, ncell_x=3, minimize=True, snapcoeff_filename='W_REF.snapcoeff', verbose=False)
print(f'Minimized energy of bcc_vacancy2: {bcc_vacancy2.energy:.8f}')

Non-minimized energy (eV): -291.66197572
Minimized energy (eV): -291.74966752
Minimized energy of bcc_vacancy2: -291.74966752


## Box relaxation
To perform the box relaxation, specify `fix_box_relax` option. For non-isotropic relaxation, in addition, specify `box_relax_iso=False` (which is `True` by default).

In [20]:
bcc_vacancy_no_box_relax = BCC_VACANCY(alat=3.2, ncell_x=2, minimize=True,
                                       snapcoeff_filename='W_REF.snapcoeff', verbose=False, comm=comm)
bcc_vacancy_with_box_relax = BCC_VACANCY(alat=3.2, ncell_x=2, minimize=True, fix_box_relax=True, box_relax_iso=True,
                                         snapcoeff_filename='W_REF.snapcoeff', verbose=False, comm=comm)

print(f'Energy of bcc_vacancy_no_box_relax: {bcc_vacancy_no_box_relax.energy:.8f}')
print(f'Energy of bcc_vacancy_with_box_relax: {bcc_vacancy_with_box_relax.energy:.8f}')
print(f'Volume of bcc_vacancy_no_box_relax: {bcc_vacancy_no_box_relax.volume:.8f}')
print(f'Volume of bcc_vacancy_with_box_relax: {bcc_vacancy_with_box_relax.volume:.8f}')
print('')
print(f'Cell of bcc_vacancy_no_box_relax: \n{bcc_vacancy_no_box_relax.cell}')
print('')
print(f'Cell of bcc_vacancy_with_box_relax: \n{bcc_vacancy_with_box_relax.cell}')

Energy of bcc_vacancy_no_box_relax: -80.05472088
Energy of bcc_vacancy_with_box_relax: -80.36528097
Volume of bcc_vacancy_no_box_relax: 262.14400000
Volume of bcc_vacancy_with_box_relax: 253.19481727

Cell of bcc_vacancy_no_box_relax: 
[[6.4 0.  0. ]
 [0.  6.4 0. ]
 [0.  0.  6.4]]

Cell of bcc_vacancy_with_box_relax: 
[[6.32632653 0.         0.        ]
 [0.         6.32632653 0.        ]
 [0.         0.         6.32632653]]
